<a href="https://colab.research.google.com/github/Himanshugawasane/FMML_Project-and-labs/blob/main/Mod1_lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In [3]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-3-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


In [4]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

In [5]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

In [6]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

In [7]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

In [8]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


In [9]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

In [10]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


In [11]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


In [12]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


1)How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?

A)Both increasing and decreasing the percentage of the validation set can have specific effects on the accuracy of the model.

More data for model evaluation: A larger validation set provides more instances for assessing the model's performance, which can lead to a more reliable estimate of generalization performance.

Better generalization: With a larger validation set, the model is tested on a broader range of examples, potentially improving its ability to generalize to new, unseen data.

2)How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?

A)The size and representativeness of the training and validation sets are crucial factors in predicting the model's accuracy on the test set. It's essential to strike a balance, ensuring that the model has enough data to learn meaningful patterns while also having a reliable estimate of its generalization performance through a well-chosen validation set.









In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


1)Does averaging the validation accuracy across multiple splits give more consistent results?

A)Yes, averaging the validation accuracy across multiple splits, often referred to as cross-validation, can provide more consistent and robust results compared to relying on a single split. Cross-validation helps mitigate the impact of the specific choice of the training and validation sets, providing a more reliable estimate of a model's performance.

2)Does it give more accurate estimate of test accuracy?

A)Cross-validation, by itself, doesn't directly estimate the test accuracy. Instead, it provides a more accurate and robust estimate of how well your model is likely to generalize to new, unseen data. The primary purpose of cross-validation is to evaluate the model's performance on different subsets of the data and reduce the impact of a specific random split between training and validation sets.

While cross-validation gives you a better estimate of how well your model might perform on unseen data compared to a single train-test split, the true test accuracy can only be determined by evaluating the model on a completely independent test set that was not used during training or validation.

3)What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?

A) while using a higher number of iterations in cross-validation can improve the stability and reliability of the estimate, it's important to consider the trade-off with computational cost. It's good practice to experiment with different values of k and observe how the estimate of model performance stabilizes as k increases, finding a balance that meets both accuracy and computational efficiency requirements.

4)Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?

A) increasing the number of iterations in cross-validation may help in obtaining a more reliable estimate of the model's performance, but it might not directly address the issue of a small training or validation dataset.







